# Oleksiy Anokhin

### May 23, 2016

## Extracting zip codes

Link: https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python

In [62]:
# Install packages
import geopy
import pandas as pd
import numpy as np
# from arcgis.geocoding import reverse_geocode
# from arcgis.geometry import Geometry
# from arcgis.gis import GIS

### Problem 1. Extract zip codes for bus stations

In [66]:
# Read bus stops data 
bus_stops = pd.read_csv('CTA_BusStops.csv')  
print(bus_stops.head())

   SYSTEMSTOP  OBJECTID                                       the_geom  \
0       11953       193   POINT (-87.54862703700002 41.72818418100002)   
1        2723       194       POINT (-87.737227163 41.749111071000016)   
2        1307       195  POINT (-87.74397362600001 41.924143016000016)   
3        6696       196   POINT (-87.65929365400001 41.86931424800002)   
4          22       197        POINT (-87.72780787099998 41.877006596)   

        STREET               CROSS_ST DIR POS ROUTESSTPG OWLROUTES     CITY  \
0  92ND STREET              BALTIMORE  EB  NS         95       NaN  CHICAGO   
1  79TH STREET  KILPATRICK (east leg)  EB  NS         79       NaN  CHICAGO   
2    FULLERTON             KILPATRICK  EB  NS         74       NaN  CHICAGO   
3       TAYLOR                 THROOP  EB  NS        157       NaN  CHICAGO   
4      JACKSON                 KARLOV  EB  FS        126       NaN  CHICAGO   

   STATUS                PUBLIC_NAM    POINT_X    POINT_Y  
0       1   92nd Str

In [69]:
# Rename columns Longitude and Latitude
bus_stops = bus_stops.rename(columns = {"POINT_X": "Longitude", "POINT_Y": "Latitude"})
print(bus_stops.head())

   SYSTEMSTOP  OBJECTID                                       the_geom  \
0       11953       193   POINT (-87.54862703700002 41.72818418100002)   
1        2723       194       POINT (-87.737227163 41.749111071000016)   
2        1307       195  POINT (-87.74397362600001 41.924143016000016)   
3        6696       196   POINT (-87.65929365400001 41.86931424800002)   
4          22       197        POINT (-87.72780787099998 41.877006596)   

        STREET               CROSS_ST DIR POS ROUTESSTPG OWLROUTES     CITY  \
0  92ND STREET              BALTIMORE  EB  NS         95       NaN  CHICAGO   
1  79TH STREET  KILPATRICK (east leg)  EB  NS         79       NaN  CHICAGO   
2    FULLERTON             KILPATRICK  EB  NS         74       NaN  CHICAGO   
3       TAYLOR                 THROOP  EB  NS        157       NaN  CHICAGO   
4      JACKSON                 KARLOV  EB  FS        126       NaN  CHICAGO   

   STATUS                PUBLIC_NAM  Longitude   Latitude  
0       1   92nd Str

In [71]:
# Check NAs
bus_stops.isnull().values.any()

True

In [72]:
# Check number of rows
print(bus_stops.info())
# We have 11074 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11074 entries, 0 to 11073
Data columns (total 14 columns):
SYSTEMSTOP    11074 non-null int64
OBJECTID      11074 non-null int64
the_geom      11074 non-null object
STREET        11074 non-null object
CROSS_ST      11074 non-null object
DIR           11074 non-null object
POS           11074 non-null object
ROUTESSTPG    11068 non-null object
OWLROUTES     2190 non-null object
CITY          11074 non-null object
STATUS        11074 non-null int64
PUBLIC_NAM    11074 non-null object
Longitude     11074 non-null float64
Latitude      11074 non-null float64
dtypes: float64(2), int64(3), object(9)
memory usage: 1.2+ MB
None


In [74]:
# Drop all NAs in columns Longitude and Latitude only, because these are columns of our interest
bus_stops = bus_stops.dropna(subset = ["Longitude", "Latitude"], how = 'all')
# Check number of rows
print(bus_stops.info())
# We have 11074 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11074 entries, 0 to 11073
Data columns (total 14 columns):
SYSTEMSTOP    11074 non-null int64
OBJECTID      11074 non-null int64
the_geom      11074 non-null object
STREET        11074 non-null object
CROSS_ST      11074 non-null object
DIR           11074 non-null object
POS           11074 non-null object
ROUTESSTPG    11068 non-null object
OWLROUTES     2190 non-null object
CITY          11074 non-null object
STATUS        11074 non-null int64
PUBLIC_NAM    11074 non-null object
Longitude     11074 non-null float64
Latitude      11074 non-null float64
dtypes: float64(2), int64(3), object(9)
memory usage: 1.3+ MB
None


In [75]:
# Keep in df only two columns
bus_stops_geo = bus_stops.iloc[:, 12:14]
print(bus_stops_geo.head())

   Longitude   Latitude
0 -87.548627  41.728184
1 -87.737227  41.749111
2 -87.743974  41.924143
3 -87.659294  41.869314
4 -87.727808  41.877007


In [76]:
# Check NAs again
bus_stops_geo.isnull().values.any()
# As we can see, there no NA rows in two columns of our interest. 

False

In [88]:
# Create a function for zip codes extraction
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']

geolocator = geopy.Nominatim(user_agent = 'my-application')

In [89]:
# Test a sample with 20 rows
test = bus_stops_geo.head(20)

In [90]:
# Extract zip codes
zipcodes = test.apply(get_zipcode, axis = 1, geolocator = geolocator, 
                               lat_field = 'Latitude', lon_field = 'Longitude')

In [91]:
print(zipcodes)

0     60617
1     60652
2     60639
3     60607
4     60644
5     60659
6     60620
7     60626
8     60610
9     60660
10    60625
11    60645
12    60628
13    60620
14    60629
15    60628
16    60644
17    60638
18    60657
19    60631
dtype: object


### Asked StackOverflow Question 
https://stackoverflow.com/questions/61990378/returning-zipcodes-for-longitude-latitude-with-geopy-avoiding-geocodertimedout

### Solution in R

**Oleksiy Anokhin**

**May 24, 2020**

**Attempt to extract zip codes in R**

Link: https://stackoverflow.com/questions/42337619/how-to-batch-reverse-geocode-in-r

library(sp)

library(rgdal)

library(tidyverse)

**Import zips shapefile and transform CRS**

zips <- readOGR("cb_2015_us_zcta510_500k.shp")

zips <- spTransform(zips, CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0"))

**Read bus stops data**

bus_stops <- read_csv("CTA_BusStops.csv")

**Rename two columns for long and lat**

bus_stops <- rename(bus_stops, longitude = POINT_X, latitude = POINT_Y)

**Extract long and lat only**
test_bus_stops <- bus_stops[ , c(13, 14)]

**Transform coordinates into a SpatialPointsDataFrame**

bus_stops_spatial <- SpatialPointsDataFrame(coords = test_bus_stops, data = bus_stops, proj4string = CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0"))

class(bus_stops_spatial) # [1] "SpatialPointsDataFrame"

**Subset only the zipcodes in which points are found**

zips_subset <- zips[bus_stops_spatial, ]

**NOTE: the column in zips_subset containing zipcodes is ZCTA5CE10**

**Use over() to overlay points in polygons and then add that to the original dataframe**

**Create a separate column**

bus_stops_zip <- over(bus_stops_spatial, zips_subset[, "zip_code"]) 

bus_stops_zip # It works! I double check long and lat via https://www.melissa.com/v2/lookups/latlngzip4/

**Merge bus stops with bus stop zips**

bus_stops <- cbind(bus_stops, bus_stops_zip)

**Rename zip_code column**

bus_stops <- rename(bus_stops, zip_code = ZCTA5CE10)

head(bus_stops)

head(bus_stops[, 13:15])

**Extract as csv**

write.csv(bus_stops,'bus_stops_zips.csv')

In [131]:
# Read bus stops data with zip codes
bus_stops_cleaned = pd.read_csv('bus_stops_zips.csv')  
print(bus_stops_cleaned.head())

   Unnamed: 0  SYSTEMSTOP  OBJECTID  \
0           1       11953       193   
1           2        2723       194   
2           3        1307       195   
3           4        6696       196   
4           5          22       197   

                                        the_geom       STREET  \
0   POINT (-87.54862703700002 41.72818418100002)  92ND STREET   
1       POINT (-87.737227163 41.749111071000016)  79TH STREET   
2  POINT (-87.74397362600001 41.924143016000016)    FULLERTON   
3   POINT (-87.65929365400001 41.86931424800002)       TAYLOR   
4        POINT (-87.72780787099998 41.877006596)      JACKSON   

                CROSS_ST DIR POS ROUTESSTPG OWLROUTES     CITY  STATUS  \
0              BALTIMORE  EB  NS         95       NaN  CHICAGO       1   
1  KILPATRICK (east leg)  EB  NS         79       NaN  CHICAGO       1   
2             KILPATRICK  EB  NS         74       NaN  CHICAGO       1   
3                 THROOP  EB  NS        157       NaN  CHICAGO       1   
4   

In [132]:
# Drop unnecessary columns for us
bus_stops_cleaned = bus_stops_cleaned[['SYSTEMSTOP','OBJECTID', 'ROUTESSTPG', 'PUBLIC_NAM', 
                                       'longitude', 'latitude', 'zip_code' ]]
print(bus_stops_cleaned.head())

   SYSTEMSTOP  OBJECTID ROUTESSTPG                PUBLIC_NAM  longitude  \
0       11953       193         95   92nd Street & Baltimore -87.548627   
1        2723       194         79  79th Street & Kilpatrick -87.737227   
2        1307       195         74    Fullerton & Kilpatrick -87.743974   
3        6696       196        157           Taylor & Throop -87.659294   
4          22       197        126          Jackson & Karlov -87.727808   

    latitude  zip_code  
0  41.728184     60617  
1  41.749111     60652  
2  41.924143     60639  
3  41.869314     60607  
4  41.877007     60624  


In [184]:
# Make all column names lowercase
bus_stops_cleaned.columns = map(str.lower, bus_stops_cleaned.columns)
print(bus_stops_cleaned.head())

   stop_id  object_id route                   address  longitude   latitude  \
0    11953        193    95   92nd Street & Baltimore -87.548627  41.728184   
1     2723        194    79  79th Street & Kilpatrick -87.737227  41.749111   
2     1307        195    74    Fullerton & Kilpatrick -87.743974  41.924143   
3     6696        196   157           Taylor & Throop -87.659294  41.869314   
4       22        197   126          Jackson & Karlov -87.727808  41.877007   

   zip_code  
0     60617  
1     60652  
2     60639  
3     60607  
4     60624  


In [185]:
# Rename several columns
bus_stops_cleaned = bus_stops_cleaned.rename(columns = {"systemstop": "stop_id", "objectid": "object_id", 
                                                        'routesstpg': 'route', 'public_nam': 'address'})
print(bus_stops_cleaned.head())

   stop_id  object_id route                   address  longitude   latitude  \
0    11953        193    95   92nd Street & Baltimore -87.548627  41.728184   
1     2723        194    79  79th Street & Kilpatrick -87.737227  41.749111   
2     1307        195    74    Fullerton & Kilpatrick -87.743974  41.924143   
3     6696        196   157           Taylor & Throop -87.659294  41.869314   
4       22        197   126          Jackson & Karlov -87.727808  41.877007   

   zip_code  
0     60617  
1     60652  
2     60639  
3     60607  
4     60624  


**Discuss with the team regarding some columns**

In [205]:
bus_stops_cleaned.to_csv("bus_stops_data.csv", index = False)

### Problem 2. Extract zip codes for L stations

In [190]:
# Read L stations data 
l_stations = pd.read_csv('CTA_-_System_Information_-_List_of__L__Stops.csv')  
print(l_stations.head())

   STOP_ID DIRECTION_ID                        STOP_NAME  \
0    30162            W         18th (54th/Cermak-bound)   
1    30161            E                18th (Loop-bound)   
2    30022            N         35th/Archer (Loop-bound)   
3    30023            S       35th/Archer (Midway-bound)   
4    30214            S  35-Bronzeville-IIT (63rd-bound)   

           STATION_NAME           STATION_DESCRIPTIVE_NAME  MAP_ID   ADA  \
0                  18th                   18th (Pink Line)   40830  True   
1                  18th                   18th (Pink Line)   40830  True   
2           35th/Archer          35th/Archer (Orange Line)   40120  True   
3           35th/Archer          35th/Archer (Orange Line)   40120  True   
4  35th-Bronzeville-IIT  35th-Bronzeville-IIT (Green Line)   41120  True   

     RED   BLUE      G    BRN      P   Pexp      Y    Pnk      O  \
0  False  False  False  False  False  False  False   True  False   
1  False  False  False  False  False  False  F

In [191]:
# Split location into long/lat
long_lat = l_stations['Location'].str.strip('()').str.split(', ', expand = True).rename(columns = {0:'latitude', 1:'longitude'}) 

In [192]:
print(long_lat)

      latitude   longitude
0    41.857908  -87.669147
1    41.857908  -87.669147
2    41.829353  -87.680622
3    41.829353  -87.680622
4    41.831677  -87.625826
..         ...         ...
295  41.964273  -87.657588
296   41.88322  -87.626189
297  41.964273  -87.657588
298  41.885269  -87.666969
299  41.885269  -87.666969

[300 rows x 2 columns]


In [193]:
# Drop location column from the dataframe and add two columns - longitude and latitude. 
print(l_stations)
l_stations.drop(l_stations.columns[len(l_stations.columns)-1], axis = 1, inplace = True)

     STOP_ID DIRECTION_ID                           STOP_NAME  \
0      30162            W            18th (54th/Cermak-bound)   
1      30161            E                   18th (Loop-bound)   
2      30022            N            35th/Archer (Loop-bound)   
3      30023            S          35th/Archer (Midway-bound)   
4      30214            S     35-Bronzeville-IIT (63rd-bound)   
..       ...          ...                                 ...   
295    30106            S                 Wilson (95th-bound)   
296    30383            N      Washington/Wabash (Outer Loop)   
297    30385            S                 Wilson (Loop-bound)   
298    30033            W  Ashland (Harlem-54th/Cermak-bound)   
299    30032            E           Ashland (Loop-63rd-bound)   

             STATION_NAME                           STATION_DESCRIPTIVE_NAME  \
0                    18th                                   18th (Pink Line)   
1                    18th                                  

In [194]:
# Print to double-check that we do not have Location anymore
print(l_stations.head())

   STOP_ID DIRECTION_ID                        STOP_NAME  \
0    30162            W         18th (54th/Cermak-bound)   
1    30161            E                18th (Loop-bound)   
2    30022            N         35th/Archer (Loop-bound)   
3    30023            S       35th/Archer (Midway-bound)   
4    30214            S  35-Bronzeville-IIT (63rd-bound)   

           STATION_NAME           STATION_DESCRIPTIVE_NAME  MAP_ID   ADA  \
0                  18th                   18th (Pink Line)   40830  True   
1                  18th                   18th (Pink Line)   40830  True   
2           35th/Archer          35th/Archer (Orange Line)   40120  True   
3           35th/Archer          35th/Archer (Orange Line)   40120  True   
4  35th-Bronzeville-IIT  35th-Bronzeville-IIT (Green Line)   41120  True   

     RED   BLUE      G    BRN      P   Pexp      Y    Pnk      O  
0  False  False  False  False  False  False  False   True  False  
1  False  False  False  False  False  False  Fal

In [197]:
# Concat l_stations and long/lat
l_stations = pd.concat([l_stations.reset_index(drop = True), long_lat], axis = 1)

In [ ]:
# Double-check the new dataframe
print(l_stations.head())

In [199]:
# Check NAs
l_stations.isnull().values.any()
# As we can see, all data is clean

False

In [203]:
# Export in csv
l_stations.to_csv('l_stations.csv', sep = ',', index = False)

### Solution in R for zip codes

In [ ]:
# Clean data in R! 